In [ ]:
import photutils as pht
from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
import astropy.units as u
import glob,os
from photutils import CircularAperture
from photutils.utils import calc_total_error
from astropy.table import Table
from astropy import table
from astropy.stats import SigmaClip
import pandas as pd


path =r"C:/Users/Alexander/Downloads/Practicas/CALIBRADAS/2022-05-13/"
#path =r"./"

final = pd.DataFrame(columns={'image','x','y','flux','flux_e'}, index=None)
for j in range(1,405):
    ma = '0'+str(j)
    
    if j <100 :
        ma = '00'+str(j)
    if j <10 :
        ma = '000'+str(j)
        
    print(ma)
    
    data=fits.getdata(path+'C_DB_TRES1b-'+ma+'_gSDSS.fit')
    
    ## here we can show some statistics about the sky
    mean, median, std = sigma_clipped_stats(data)

    ## or first mask sources then estimate the sky background
    mask = pht.make_source_mask(data, 3, 5)
    mean, median, std = sigma_clipped_stats(data, sigma=3.0, mask=mask)
    
    ## get 2D sky map
    sigma_clip = SigmaClip(sigma=3.)
    bkg_estimator = pht.SExtractorBackground()
    bkg = pht.Background2D(data, (64, 64), mask=mask,filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
     ## find objects and calculate basic information
    daofind = pht.IRAFStarFinder(fwhm=4.5, threshold=5.*bkg.background_rms_median,exclude_border=True, sharplo=
    0.5, sharphi=2.0, roundlo=0.0, roundhi=0.7)
    sources = daofind(data - bkg.background_median)
    
    x_min = 990
    x_max = 1050
    y_min = 950
    y_max = 970
    if j <=200:
        x_min = 1040
        x_max = 1080
        y_min = 950
        y_max = 990
    if j>200 and j <=300:
        x_min = 1000
        x_max = 1050
        y_min = 950
        y_max = 990
    if j>300 and j <=500:
        x_min = 990
        x_max = 1020
        y_min = 950
        y_max = 980
        
    for t in sources:
        if t['xcentroid'] > x_min and t['xcentroid'] <= x_max and t['ycentroid'] > y_min and t['ycentroid'] <= y_max:
            sources = t
    print(sources)
    if(len(sources)/12 >1):
        print(' MAYOR QUE 1 ' + str(len(sources)))
        positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
        apertures = CircularAperture(positions, r=8)
        plt.figure(figsize=(8,8))
        plt.imshow(data, cmap='Greys_r', origin='lower', vmin=300,vmax=600, interpolation='nearest')
        apertures.plot(color='red', lw=1.5, alpha=0.5)
    else:
        positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
        apertures = CircularAperture(positions, r=8)
        plt.figure(figsize=(8,8))
        plt.imshow(data, cmap='Greys_r', origin='lower', vmin=300,vmax=600, interpolation='nearest')
        apertures.plot(color='red', lw=1.5, alpha=0.5)

        error=calc_total_error(data-bkg.background, bkg.background_rms, 1.85)

        radii=[10]
        positions=[sources['xcentroid'],sources['ycentroid']]
        apertures = [pht.CircularAperture(positions, r=r) for r in radii]
        aper_phot= pht.aperture_photometry(data - bkg.background, apertures, error=error)
        print(aper_phot)
        final = final.append({'image':ma,'x':aper_phot[0]['xcenter'],'y':aper_phot[0]['ycenter'],'flux':aper_phot[0]['aperture_sum'], 'flux_e':aper_phot[0]['aperture_sum_err']}, ignore_index=True)

#final.to_csv('resultados_ESTANDAR.csv')
print(final)